In [1]:
! pip install transformers

In [2]:
! pip install pandas_gbq
import pandas_gbq

from google.cloud import bigquery
bqclient = bigquery.Client()

project_id = "positube-389309"
# dataset_id = "comments_dataset_dislikes"
# table_id = "video_score_test"

In [3]:
import pandas as pd
import numpy as np
import requests
import os
import tensorflow as tf

2023-06-09 16:04:11.458225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 16:04:12.372777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-09 16:04:12.372910: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [4]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [5]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

2023-06-09 16:04:16.902535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 16:04:16.914883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 16:04:16.916600: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-09 16:04:16.918684: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
BIGdf = pd.read_csv('final_df_CLEAN.csv', index_col=0)

In [7]:
# BIGdf = BIGdf[0:500]

In [8]:
BIGdf

,comment,author,likecount,date,replies,video_id
0,Respect to Dortmund fans must be sad losing him.,Uh Idk,1932,2021-07-01T10:33:16Z,59,--0bCF-iK2E
1,A real talent hope he doesn't turn into anoth...,7h35h96u9,617,2021-07-01T10:28:25Z,34,--0bCF-iK2E
2,wish him well in manchester united,Guncius,2583,2021-07-01T10:01:00Z,94,--0bCF-iK2E
3,One of the most talented players Bundesliga ha...,Rishi Joe Sanu,57,2021-07-01T12:46:06Z,1,--0bCF-iK2E
4,What a spell in the Bundesliga Goodluck in th...,H2KAL5859,66,2021-07-01T10:01:31Z,0,--0bCF-iK2E
...,...,...,...,...,...,...
495,Ye killed it! The spiritual mix in music is “h...,Jay Bee,2,2021-08-06T19:50:49Z,0,-0PZSxZuAXQ
496,An AMAZING EMOTIONAL ENCYCLOPEDIA OF SOME Of T...,RoseBud20,0,2021-08-07T22:21:50Z,0,-0PZSxZuAXQ
497,Facts or undeniable facts?...Kanye West needs ...,The Alkebulan Trust,46,2021-08-06T12:11:17Z,2,-0PZSxZuAXQ
498,Hate him or love him he is the greatest artist...,Bryan Kicks,0,2021-08-06T20:21:12Z,0,-0PZSxZuAXQ


In [9]:
BIGdf['video_id']

0      --0bCF-iK2E
1      --0bCF-iK2E
2      --0bCF-iK2E
3      --0bCF-iK2E
4      --0bCF-iK2E
          ...     
495    -0PZSxZuAXQ
496    -0PZSxZuAXQ
497    -0PZSxZuAXQ
498    -0PZSxZuAXQ
499    -0PZSxZuAXQ
Name: video_id, Length: 500, dtype: object

In [10]:
BIGdf.value_counts('video_id').keys()

Index(['--0bCF-iK2E', '--DKkzWVh-E', '-024Swollbc', '-0PZSxZuAXQ'], dtype='object', name='video_id')

In [11]:
likes = np.nan

In [12]:
def making_weights(num):
    '''This function makes weights for each comment based on its like count (num)'''
    if num == 0:
        return 1
    elif num > 0 and num <= np.median(likes[:len(likes)//2]):
        return 2
    elif num > np.median(likes[:len(likes)//2]) and num <= np.median(likes):
        return 3
    elif num > np.median(likes) and num < np.median(likes[len(likes)//2:]):
        return 4
    else:
        return 5

In [13]:
def df_cutter(df):
    IDs_list = df.value_counts('video_id').keys()
    cut_dfs = []
    for i, video in enumerate(IDs_list):
        cut_df = df[df['video_id'] == IDs_list[i]].copy()
        global likes
        likes = sorted(list(cut_df[cut_df['likecount'] > 0]['likecount']))
        cut_df['weight'] = cut_df['likecount'].apply(making_weights)
        cut_dfs.append(cut_df.copy())
    return cut_dfs

In [14]:
cut_dfs = df_cutter(BIGdf)

In [15]:
def sentiment_score_comment(df):

    '''This function predicts the sentiment score of each youtube video!'''
    
    model_name = "cardiffnlp/twitter-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

    # Lists to store the sentiment analysis results
    sentiment_list = []
    negative_list = []
    neutral_list = []
    positive_list = []
    scalar_value_list = []
    weighted_SV = []
    weight = list(df['likecount'].apply(making_weights))

    # Iterate over the comments in the DataFrame
    for i, text in enumerate(df['comment']):
        
        # Tokenization, Sentiment Prediction, and Interpretation
        tokens = tokenizer.encode_plus(text, add_special_tokens=True, padding='longest', truncation=True, max_length=512, return_tensors='tf')
        outputs = model(tokens.input_ids)
        logits = outputs.logits
        prediction = np.array(tf.nn.softmax(logits)[0])
        predicted_class = tf.argmax(logits, axis=1).numpy()[0]
        sentiment_labels = ["Negative", "Neutral", "Positive"]
        predicted_sentiment = sentiment_labels[predicted_class]

        # Append the sentiment analysis results to the respective lists
        sentiment_list.append(predicted_sentiment)
        negative_list.append(round(prediction[0]*100, 2))
        neutral_list.append(round(prediction[1]*100, 2))
        positive_list.append(round(prediction[2]*100, 2))
        scalar_value_val = round((prediction[0])*-1+(prediction[2]*1),2)
        scalar_value_list.append(scalar_value_val)
        weighted_SV.append(df['weight'].iloc[i] * scalar_value_val)


    # Create a new DataFrame with the sentiment analysis results
    results_df = pd.DataFrame({
        'Comment': df['comment'],
        'Sentiment': sentiment_list,
        'Negative_percent': negative_list,
        'Neutral_percent': neutral_list,
        'Positive_percent': positive_list,
        'Scaler_value': scalar_value_list,
        'weighted_SV': weighted_SV,
        'weight': weight
    })

    # Return the new DataFrame
    return results_df


In [16]:
results_list = []
for idx, df in enumerate(cut_dfs):
    results = sentiment_score_comment(df)
    results['video_id'] = df['video_id']
    results_list.append(results)
    print('I DID THIS VIDEO ', idx)
    pandas_gbq.to_gbq(results, f'sentiment_scores.sentiment_up_to_{idx}', project_id=project_id)
    print('AND NOW IT IS ON BIG QUERY', '\n----------\n')
results_list

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


I DID THIS VIDEO  0


100%|██████████| 1/1 [00:00<00:00, 8035.07it/s]


AND NOW IT IS ON BIG QUERY 
----------



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


...

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


I DID THIS VIDEO  3


100%|██████████| 1/1 [00:00<00:00, 9098.27it/s]

AND NOW IT IS ON BIG QUERY 
----------



[                                               Comment Sentiment  \
 0     Respect to Dortmund fans must be sad losing him.   Neutral   
 1    A real talent  hope he doesn't turn into anoth...   Neutral   
 2                   wish him well in manchester united  Positive   
 3    One of the most talented players Bundesliga ha...  Positive   
 4    What a spell in the Bundesliga  Goodluck in th...  Positive   
 ..                                                 ...       ...   
 170         Has potential to be the next Ronaldo/messi  Positive   
 171                              Sancho <3  Thank you.  Positive   
 172  Welcome to Manchester United the theatre of dr...  Positive   
 173  I hope he became much stronger and healthier i...  Positive   
 174  All i can say is  good luck on your next desti...  Positive   
 
      Negative_percent  Neutral_percent  Positive_percent  Scaler_value  \
 0               33.80            50.22             15.98         -0.18   
 1               10.

In [17]:
results_list[0]

,Comment,Sentiment,Negative_percent,Neutral_percent,Positive_percent,Scaler_value,weighted_SV,weight,video_id
0,Respect to Dortmund fans must be sad losing him.,Neutral,33.80,50.22,15.98,-0.18,-0.90,5,--0bCF-iK2E
1,A real talent hope he doesn't turn into anoth...,Neutral,10.69,63.89,25.42,0.15,0.75,5,--0bCF-iK2E
2,wish him well in manchester united,Positive,0.31,11.78,87.91,0.88,4.40,5,--0bCF-iK2E
3,One of the most talented players Bundesliga ha...,Positive,0.22,1.68,98.10,0.98,4.90,3,--0bCF-iK2E
4,What a spell in the Bundesliga Goodluck in th...,Positive,0.18,3.56,96.26,0.96,4.80,4,--0bCF-iK2E
...,...,...,...,...,...,...,...,...,...
170,Has potential to be the next Ronaldo/messi,Positive,0.86,35.08,64.07,0.63,0.63,1,--0bCF-iK2E
171,Sancho <3 Thank you.,Positive,0.21,7.30,92.49,0.92,3.68,2,--0bCF-iK2E
172,Welcome to Manchester United the theatre of dr...,Positive,0.17,6.92,92.91,0.93,1.86,2,--0bCF-iK2E
173,I hope he became much stronger and healthier i...,Positive,0.14,2.31,97.56,0.97,1.94,2,--0bCF-iK2E


In [18]:
comment_score_df = pd.concat(results_list)

In [19]:
IDs_df = pd.DataFrame(BIGdf.value_counts('video_id').keys())

In [20]:
IDs_df['positivity_score'] = np.nan

In [21]:
IDs_df

,video_id,positivity_score
0,--0bCF-iK2E,NaN
1,--DKkzWVh-E,NaN
2,-024Swollbc,NaN
3,-0PZSxZuAXQ,NaN


In [22]:
for i, df in enumerate(cut_dfs):
    
    df['weight'] = df['likecount'].apply(making_weights)
    
    score_df = sentiment_score_comment(df)
    
    positivity_score = score_df['weighted_SV'].mean()
    
    IDs_df['positivity_score'][i] = positivity_score
    
    pandas_gbq.to_gbq(IDs_df, f'video_positivity_scores.positivity_up_to_{i}', project_id=project_id)
    
    print('I AM AS FAR AS VIDEO ', i, ' that is ', (i/len(cut_dfs))*100, '%\n----------\n')

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/var/tmp/ipykernel_16757/3420579272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IDs_df['positivity_score'][i] = positivity_score
100%|██████████| 1/1 [00:00<00:00, 10699.76it/s]


I AM AS FAR AS VIDEO  0  that is  0.0 %
----------



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/var/tmp/ipykernel_16757/3420579272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IDs_df['positivity_score'][i] = positivity_score
100%|██████████| 1/1 [00:00<00:00, 10131.17it/s]


I AM AS FAR AS VIDEO  1  that is  25.0 %
----------



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/var/tmp/ipykernel_16757/3420579272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IDs_df['positivity_score'][i] = positivity_score
100%|██████████| 1/1 [00:00<00:00, 11491.24it/s]


I AM AS FAR AS VIDEO  2  that is  50.0 %
----------



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
/var/tmp/ipykernel_16757/3420579272.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IDs_df['positivity_score'][i] = positivity_score
100%|██████████| 1/1 [00:00<00:00, 11008.67it/s]

I AM AS FAR AS VIDEO  3  that is  75.0 %
----------



In [23]:
results = sentiment_score_comment(df)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [24]:
def df_to_csv(results_df):
    IDs_df.to_csv('video_score.csv')
    comment_score_df.to_csv('comment_score.csv')
    
    

In [25]:
df_to_csv(IDs_df)

In [26]:
# video_id = IDs_df.iloc[0].video_id
# IDs_df.to_csv(f'video_score{video_id}.csv')

In [27]:
# IDs_df